# Day 13

## Part 1

- compare 2 signals
- first is `left` then `right`
- if both values are `int`:
    - the smaller value should be first
- if both values are `list`:
    - compare each value in turn as above
    - if `left` is shorter than `right` this is correct
- if one is `int` and one is `list` convert the `int` to a single item `list` and compare
- packets are enumerated from 1

`Determine which pairs of packets are already in the right order. What is the sum of the indices of those pairs?`

In [24]:
from utils import parse_from_file, ParseConfig

def parse_recursive_lists(string: str) -> 'list[list]':
    """
    returns a multi-dimensional list object of the string passed
    """
    output = list()
    if string == '':
        return output
    segments = string.split(',')
    skip_forward = 0
    for index, segment in enumerate(segments):
        if skip_forward != 0:
            skip_forward -= 1
            continue
        if segment.startswith('['):
            start = len(''.join(segments[:index])) + index
            # print('start', start, string[:start], string[start])
            level = 0
            end = start
            while (level > 0 or end == start) and end < len(string) - 1:
                # print(level, end, string[end])
                if string[end] == '[':
                    level += 1
                elif string[end] == ']':
                    level -= 1
                end += 1
                # print(level)
            substring = string[start+1:end-1+level]
            # print('substring', substring, level, start, end)
            skip_forward = len(substring.split(',')) - 1
            output.append(parse_recursive_lists(substring))
        else:
            output.append(int(segment.strip(']')))
    return output


def list_parser_entry(string: str) -> list[list]:
    """
    entry point for the recursive parser - removes one list layer at the end
    """
    return parse_recursive_lists(string)[0]


test_string = '[[10,[],[9,8,6,[2,7,0,8]],[],[[4,6,2,1,4],6,5]]]'
print(list_parser_entry(test_string))


[[10, [], [9, 8, 6, [2, 7, 0, 8]], [], [[4, 6, 2, 1, 4], 6, 5]]]


In [25]:

parser = ParseConfig('\n\n', ParseConfig('\n', list_parser_entry))

signal_pairs = parse_from_file('day_13.txt', parser)

print('\n\n'.join([str(pair) for pair in signal_pairs[:3]]))

[[[7, 6, 3, 6, 10], [[[5, 3, 6, 7, 7]], 2, 1, 2, 6]], [[], [8, [8, 6, 5], [[5, 1, 8]], 7], []]]

[[[10, [], [9, 8, 6, [2, 7, 0, 8]], [], [[4, 6, 2, 1, 4], 6, 5]]], [[4, 5, 1, [[0, 0], [6, 10], 4, 5], 10], [[2, 4, 6, 1, [6, 8]], [[2, 0, 10]], [[5, 10], [6, 4, 5, 6], [8, 9, 2]]], [[[], 10, 0, 7, 7], [[9, 9, 7, 3, 8]]], [[[7], 3, 8], [6, 3], 1, 5]]]

[[[0, [[9, 6, 4, 5]], []]], [[10, 9, [[5, 7], 4, [5, 8, 0, 9, 8], [0, 10], [3, 1]], 4]]]


In [26]:
# now that the signals are in a nice format we can check them
def check_signal_pair(signal_pair: 'list[list]') -> bool|None:
    """
    returns true/false if the signal order has been confirmed, otherwise None
    """
    left, right = signal_pair
    # pop integer values into lists
    if isinstance(left, int):
        left = [left]
    if isinstance(right, int):
        right = [right]
    # iterate through each item in the lists
    for left_item, right_item in zip(left, right):
        # if at least one list is present
        if isinstance(left_item, list) or isinstance(right_item, list):
            result = check_signal_pair((left_item, right_item))
            if result is None:
                continue
            else:
                return result
        # else we must have 2 integers
        if left_item == right_item:
                continue
        else:
            return left_item < right_item
    # if we get this far then at least one of the lists has finished
    if len(left) == len(right):
        return None
    else:
        return len(left) < len(right)

In [28]:
sum_of_correct_indecies = sum(
    [index for index, pair in
    enumerate(signal_pairs, 1) if check_signal_pair(pair)
])

print(f'The sum of indecies of correct signals is: {sum_of_correct_indecies}')

The sum of indecies of correct signals is: 5390
